---
<center>

  # **Tarea 07: Efecto Zeeman**

**Realizado Por:**

   Samuel Huertas Rojas

---
</center>

# 1 
Cálcule el efecto Zeeman para el átomo de hidrógeno usando un método variacional. Utilice como base los autoestados del átomo de hidrógeno no relativista para $n=2$ y $\ell=1$ y 100 valores de $B$ entre $0.001$ y $2$ Teslas

# 2
Compare las soluciones perturbativas con el método variacional para $B=0.04$ y $5$ Teslas de nuevo para $n=2$, $\ell=1$

# 3
Incluya el segundo orden para la perturbación de $SO$ y compruebe cómo se mejora el resuldato de la aproximación de Paschen-Back para $B=5$ Teslas